In [1]:
import numpy as np
import pandas as pd
import emoji

from keras.models import Sequential
from keras.layers import Dense, Input, Dropout, SimpleRNN,LSTM, Activation, Bidirectional
from keras.utils import np_utils

import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
train = pd.read_csv('dataset/train_emoji.csv',header=None)
test = pd.read_csv('dataset/test_emoji.csv',header=None)

train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [3]:
import emoji


emoji_dictionary = {"0": "\u2764\uFE0F",    # :heart: prints a black instead of red heart depending on the font
                    "1": ":baseball:",
                    "2": ":grinning_face_with_big_eyes:",
                    "3": ":disappointed_face:",
                    "4": ":fork_and_knife:",
                    "5": ":hundred_points:",
                    "6": ":fire:",
                    "7": ":face_blowing_a_kiss:",
                    "8": ":chestnut:",
                    "9":":flexed_biceps:"
                   }


In [4]:
#emoji.EMOJI_UNICODE

In [5]:
for e in emoji_dictionary.values():
    print(emoji.emojize(e))

❤️
⚾
😃
😞
🍴
💯
🔥
😘
🌰
💪


In [6]:
data  = train.values
for i in range(5):
    print(data[i][0],emoji.emojize(emoji_dictionary[str(data[i][1])]))
    

never talk to me again 😞
I am proud of your achievements 😃
It is the worst day in my life 😞
Miss you so much ❤️
food is life 🍴


In [7]:
print(data.shape)

(132, 4)


In [8]:
X_train = train[0]
Y_train = train[1]

X_test = test[0]
Y_test = test[1]

print(X_test.shape)

(56,)


In [9]:

for ix in range(X_train.shape[0]):
    X_train[ix] = X_train[ix].split()
    
for ix in range(X_test.shape[0]):
    X_test[ix] = X_test[ix].split()

c:\python36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\python36\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [10]:
print(X_train.shape)

(132,)


In [11]:
Y_train = np_utils.to_categorical(Y_train)
        

In [12]:
Yt = np_utils.to_categorical(Y_test)

In [13]:
print(X_train[0],Y_train[0])

['never', 'talk', 'to', 'me', 'again'] [0. 0. 0. 1. 0.]


In [14]:
embeddings_index = {}
f = open('glove.6B.50d.txt',encoding='utf-8')

In [15]:
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:],dtype='float')
    embeddings_index[word] = coefs
f.close()

In [16]:
embeddings_index['food'].shape

(50,)

In [17]:
def getEmbeddingMatrix(sentences):
    embedding_matrix = np.zeros((X_train.shape[0],10,50))
    
    for ix in range(sentences.shape[0]):
        for ij in range(len(sentences[ix])):
            embedding_matrix[ix][ij] = embeddings_index[X_train[ix][ij].lower()]
    
    return embedding_matrix

In [18]:
embedding_matrix_train = np.zeros((X_train.shape[0], 10, 50))
embedding_matrix_test = np.zeros((X_test.shape[0], 10, 50))

for ix in range(X_train.shape[0]):
    for ij in range(len(X_train[ix])):
        embedding_matrix_train[ix][ij] = embeddings_index[X_train[ix][ij].lower()]
        
for ix in range(X_test.shape[0]):
    for ij in range(len(X_test[ix])):
        try:
            embedding_matrix_test[ix][ij] = embeddings_index[X_test[ix][ij].lower()]   
        except:
            embedding_matrix_test[ix][ij] = np.zeros((50,))

In [19]:
print(embedding_matrix_train.shape,embedding_matrix_test.shape)

(132, 10, 50) (56, 10, 50)


In [20]:
model = Sequential()

#model.add(LSTM(64, input_shape=(10,50), return_sequences=True,recurrent_dropout=0.1))
#model.add(Dropout(0.5))
model.add(LSTM(64,input_shape=(10,50), return_sequences=False,recurrent_dropout=0.1))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 64)                29440     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 325       
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
Total params: 29,765
Trainable params: 29,765
Non-trainable params: 0
_________________________________________________________________


In [21]:
from keras.callbacks import ModelCheckpoint # save the best model, fight overfiitting
from keras.callbacks import EarlyStopping #save time


checkpoint = ModelCheckpoint("best_model.h5", monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
earlystop = EarlyStopping(monitor='val_acc',patience=1)



earlystop = EarlyStopping(patience=10)

hist = model.fit(embedding_matrix_train,Y_train,
                epochs = 200, batch_size=32,shuffle=True,validation_split=0.1,callbacks=[checkpoint,earlystop])

Train on 118 samples, validate on 14 samples
Epoch 1/200
118/118 [==============================] - 1s 10ms/step - loss: 1.6110 - acc: 0.2373 - val_loss: 1.5842 - val_acc: 0.3571
Epoch 2/200
118/118 [==============================] - 0s 321us/step - loss: 1.5642 - acc: 0.4068 - val_loss: 1.5932 - val_acc: 0.2143
Epoch 3/200
118/118 [==============================] - 0s 406us/step - loss: 1.5473 - acc: 0.3898 - val_loss: 1.6026 - val_acc: 0.2143
Epoch 4/200
118/118 [==============================] - 0s 397us/step - loss: 1.5190 - acc: 0.4153 - val_loss: 1.6072 - val_acc: 0.2143
Epoch 5/200
118/118 [==============================] - 0s 372us/step - loss: 1.4896 - acc: 0.4153 - val_loss: 1.6049 - val_acc: 0.2143
Epoch 6/200
118/118 [==============================] - 0s 296us/step - loss: 1.4576 - acc: 0.4407 - val_loss: 1.6066 - val_acc: 0.2143
Epoch 7/200
118/118 [==============================] - 0s 363us/step - loss: 1.4452 - acc: 0.3983 - val_loss: 1.5948 - val_acc: 0.2143
Epoch 8/200

In [22]:
pred = model.predict_classes(embedding_matrix_test)

In [23]:
print(pred)

[4 3 2 3 2 2 3 2 4 2 1 2 3 3 1 3 3 2 3 2 3 0 4 0 3 3 2 0 4 2 0 1 3 2 0 1 2
 3 4 2 1 0 0 1 2 2 3 2 3 1 3 0 3 2 2 2]


In [24]:
t = np.array(test[1])

In [25]:
float(sum(pred==t))/embedding_matrix_test.shape[0]

0.625

In [26]:
model.load_weights("best_model.h5")

In [27]:
pred = model.predict_classes(embedding_matrix_test)
t = np.array(test[1])
float(sum(pred==t))/embedding_matrix_test.shape[0]

0.625

In [28]:
test_sen = "it is a great movie!"

In [29]:
for w in test_sen.
embeddings = 

SyntaxError: invalid syntax (<ipython-input-29-e4c08fa49f1e>, line 1)

In [ ]:
print(test.shape)